## Transorming results data and storing it in silver layer for future transformations

####  using run commands to access notebook to run codes to configuring the adlsgen2 and connect

In [0]:
%run "../includes/adlsconfig"

In [0]:
%run "../includes/containersconfig"

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
results_struct_schema = StructType([
                                     StructField("constructorId", LongType(), True),
                                     StructField("driverId", LongType(), True),
                                     StructField("fastestLap", StringType(), True),
                                     StructField("fastestLapSpeed", StringType(), True),
                                     StructField("fastestLapTime", StringType(), True),
                                     StructField("grid", LongType(), True),
                                     StructField("laps", LongType(), True),
                                     StructField("milliseconds", StringType(), True),
                                     StructField("number", StringType(), True),
                                     StructField("points", DoubleType(), True),
                                     StructField("position", StringType(), True),
                                     StructField("positionOrder", LongType(), True),
                                     StructField("positionText", StringType(), True),
                                     StructField("raceId", LongType(), True),
                                     StructField("rank", StringType(), True),
                                     StructField("resultId", LongType(), True),
                                     StructField("statusId", LongType(), True),
                                     StructField("time", StringType(), True),

])

In [0]:
results_df = spark.read.format('json').option('header', True).schema(results_struct_schema).load(f"{raw_folder_path}/results.json")

In [0]:

results_final_df = results_df.drop(col("statusId"))\
    .withColumnsRenamed({"resultId": "result_id", "raceId": "race_id", "driverId" : "driver_id", "constructorId" : "constructor_id", "positionText" : "position_text", "positionOrder" : "position_order", "fastestLap" : " fastest_lap", "fastestLapTime" : "fastest_lap_time", "fastestLapSpeed" : "fastest_lap_speed"})\
        .withColumn("ingestion_date", current_timestamp())


In [0]:
results_final_df.display()

In [0]:
results_final_df.write.format('parquet').mode('overwrite').partitionBy('race_id').option('path',f"{silver_folder_path}/results").save()

In [0]:
spark.read.format('parquet').option('header', True).option('inferSchema', True).load(f"{silver_folder_path}/results").display()